In [6]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import torchvision
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn
from torch.autograd import Variable
from torch.cuda.amp import autocast, GradScaler
from torchvision import datasets, transforms, models
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import os
import time

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(5, shear=10, scale=(0.8, 1.2)),
    transforms.ColorJitter(brightness=0.5, contrast=0.6, saturation=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

training_dataset = datasets.ImageFolder('train', transform=transform_train)
validation_dataset = datasets.ImageFolder('val', transform=transform)

training_loader = torch.utils.data.DataLoader(training_dataset, batch_size=64, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=64, shuffle=False)

In [4]:
def im_convert(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
    image = tensor.cpu().clone().detach().numpy()
    image = image.transpose(1, 2, 0)
    image = image * np.array(std) + np.array(mean)
    image = image.clip(0, 1)
    return image

In [5]:
classes = ('cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash')

In [28]:
class MobileNetV2SVM(nn.Module):
    def __init__(self, num_classes=6):
        super(MobileNetV2SVM, self).__init__()
        self.mobilenet = models.mobilenet_v2(pretrained=True)
        self.feature_extractor = nn.Sequential(*list(self.mobilenet.children())[:-1])
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(1280, num_classes)  # Adjust input size according to MobileNetV2's output features
        self.svm = nn.Linear(num_classes, num_classes)  # SVM layer
        
    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = self.svm(x)
        return x

In [29]:
# Create an instance of MobileNetV2SVM
model = MobileNetV2SVM(num_classes=6)
model.to(device)

MobileNetV2SVM(
  (mobilenet): MobileNetV2(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 96, kernel_si

In [30]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [36]:
# Define SVM classifier
svm = SVC(kernel='linear', C=100)

In [37]:
# Train SVM classifier
train_features = []
train_labels = []

# Extract features from training data
with torch.no_grad():
    for inputs, labels in training_loader:
        inputs = inputs.to(device)
        features = mobilenet(inputs).squeeze().cpu().numpy()
        train_features.append(features)
        train_labels.append(labels.numpy())

train_features = np.concatenate(train_features, axis=0)
train_labels = np.concatenate(train_labels, axis=0)

# Scale features
scaler = StandardScaler()
scaled_train_features = scaler.fit_transform(train_features)

In [38]:
# Train SVM classifier
svm.fit(scaled_train_features, train_labels)

SVC(C=100, kernel='linear')

In [39]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Validate SVM classifier
validation_features = []
validation_labels = []

# Extract features from validation data
with torch.no_grad():
    for inputs, labels in validation_loader:
        inputs = inputs.to(device)
        features = mobilenet(inputs).squeeze().cpu().numpy()
        validation_features.append(features)
        validation_labels.append(labels.numpy())

validation_features = np.concatenate(validation_features, axis=0)
validation_labels = np.concatenate(validation_labels, axis=0)

# Scale validation features
scaled_validation_features = scaler.transform(validation_features)

# Evaluate SVM classifier
validation_accuracy = svm.score(scaled_validation_features, validation_labels)
print("Validation Accuracy:", validation_accuracy)

# Calculate precision, recall, and F1-score
predicted_labels = svm.predict(scaled_validation_features)
precision = precision_score(validation_labels, predicted_labels, average='weighted')
recall = recall_score(validation_labels, predicted_labels, average='weighted')
f1score = f1_score(validation_labels, predicted_labels, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1score)

Validation Accuracy: 0.7773386034255599
Precision: 0.7829851245445641
Recall: 0.7773386034255599
F1-score: 0.7773807529389501


In [40]:
# Train SVM classifier for multiple epochs
num_epochs = 5

for epoch in range(num_epochs):
    train_features = []
    train_labels = []

    # Extract features from training data
    with torch.no_grad():
        for inputs, labels in training_loader:
            inputs = inputs.to(device)
            features = mobilenet(inputs).squeeze().cpu().numpy()
            train_features.append(features)
            train_labels.append(labels.numpy())

    train_features = np.concatenate(train_features, axis=0)
    train_labels = np.concatenate(train_labels, axis=0)

    # Scale features
    scaler = StandardScaler()
    scaled_train_features = scaler.fit_transform(train_features)

    # Train SVM classifier
    svm.fit(scaled_train_features, train_labels)

    # Validate SVM classifier
    validation_features = []
    validation_labels = []

    # Extract features from validation data
    with torch.no_grad():
        for inputs, labels in validation_loader:
            inputs = inputs.to(device)
            features = mobilenet(inputs).squeeze().cpu().numpy()
            validation_features.append(features)
            validation_labels.append(labels.numpy())

    validation_features = np.concatenate(validation_features, axis=0)
    validation_labels = np.concatenate(validation_labels, axis=0)

    # Scale validation features
    scaled_validation_features = scaler.transform(validation_features)

    # Evaluate SVM classifier
    validation_accuracy = svm.score(scaled_validation_features, validation_labels)
    print(f"Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {validation_accuracy}")

KeyboardInterrupt: 